In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis = 1)
application_df

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0                  T10       Independent          C1000    ProductDev   
1                   T3       Independent          C2000  Preservation   
2                   T5  CompanySponsored          C3000    ProductDev   
3                   T3  CompanySponsored          C2000  Preservation   
4                   T3       Independent          C1000     Heathcare   
...                ...               ...            ...           ...   
34294               T4       Independent          C1000    ProductDev   
34295               T4  CompanySponsored          C3000    ProductDev   
34296               T3  CompanySponsored          C2000  Preservation   
34297               T5       Independent          C3000    ProductDev   
34298               T3       Independent          C1000  Preservation   

       ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS   ASK_AMT  \
0       Association       1              0                      N      5000   
1      Co-operative       1         1-9999                      N    108590   
2       Association       1              0                      N      5000   
3             Trust       1    10000-24999                      N      6692   
4             Trust       1  100000-499999                      N    142590   
...             ...     ...            ...                    ...       ...   
34294   Association       1              0                      N      5000   
34295   Association       1              0                      N      5000   
34296   Association       1              0                      N      5000   
34297   Association       1              0                      N      5000   
34298  Co-operative       1          1M-5M                      N  36500179   

       IS_SUCCESSFUL  
0                  1  
1                  1  
2                  0  
3                  1  
4                  1  
...              ...  
34294              0  
34295              0  
34296              0  
34297              1  
34298              0  

[34299 rows x 10 columns]

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C1260',
                              'C2710',
                              'C0',
                              'C3200',
                              'C1234',
                              'C1246',
                              'C1267',
                              'C1256',
                              'C2190',
                              'C4200',
                              'C2600',
                              'C5200',
                              'C1370',
                              'C1248',
                              'C6100',
                              'C1820',
                              'C1900',
                              'C1236',
                              'C3700',
                              'C2570',
                              'C1580',
                              'C1245',
                              'C2500',
                              'C1570',
                              'C1283',
                              'C2380',
                              'C1732',
                              'C1728',
                              'C2170',
                              'C4120',
                              'C8210',
                              'C2561',
                              'C4500',
                              'C2150',
                              'C1235',        
                              'C1237',        
                              'C7210',        
                              'C4100',        
                              'C1720',        
                              'C2400',        
                              'C1257',        
                              'C1600']
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [5]:
bins = [0, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000, 10000000]
application_df['BIN_ASK'] = pd.cut(application_df['ASK_AMT'], bins)
#application_df
application_df['BIN_ASK'].value_counts()

(0, 5000]              25398
(10000, 50000]          2398
(100000, 500000]        2304
(50000, 100000]         1423
(1000000, 5000000]       940
(500000, 1000000]        650
(5000, 10000]            549
(5000000, 10000000]      225
Name: BIN_ASK, dtype: int64

In [6]:
clmns = ["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","SPECIAL_CONSIDERATIONS","INCOME_AMT","BIN_ASK"]
final_df = pd.get_dummies(application_df, columns = clmns)
final_df = final_df.drop("ASK_AMT",axis = 1)
final_df.columns

Index(['STATUS', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T12', 'APPLICATION_TYPE_T13',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T2', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8', 'APPLICATION_TYPE_T9',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C1230', 'CLASSIFICATION_C1238', 'CLASSIFICATION_C1240',
       'CLASSIFICATION_C1250', 'CLASSIFICATION_C1270', 'CLASSIFICATION_C1278',
       'CLASSIFICATION_C1280', 'CLASSIFICATION_C1300', 'CLASSIFICATION_C1400',
       'CLASSIFICATION_C1500', 'CLASSIFICATION_C1700', 'CLASSIFICATION_C1800',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C2300',
       'CLA

In [7]:
# Split our preprocessed data into our features and target arrays
y = final_df["IS_SUCCESSFUL"]
X = final_df.drop("IS_SUCCESSFUL",axis = 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=18)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
    
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=234, activation="tanh", input_dim=78))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=78, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=9, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 234)               18486     
                                                                 
 dense_1 (Dense)             (None, 78)                18330     
                                                                 
 dense_2 (Dense)             (None, 9)                 711       
                                                                 
 dense_3 (Dense)             (None, 1)                 10        
                                                                 
Total params: 37,537
Trainable params: 37,537
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5647 - accuracy: 0.7205
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7330
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7341
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5429 - accuracy: 0.7348
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7367
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accuracy: 0.7374
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7382
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7395
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5115 - accuracy: 0.7490
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5107 - accuracy: 0.7493
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5109 - accuracy: 0.7488
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5109 - accuracy: 0.7491
Epoch 85/100
804/804 [==============================] - 1s 1000us/step - loss: 0.5109 - accuracy: 0.7483
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5106 - accuracy: 0.7491
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5105 - accuracy: 0.7493
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5111 - accuracy: 0.7487
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5110 - accuracy: 0.7496
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5106 - accurac

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6178 - accuracy: 0.7209 - 351ms/epoch - 1ms/step
Loss: 0.6178440451622009, Accuracy: 0.7209329605102539


# Summary

### This analysis looked at whether or not a charity was succesful based off of features such as organization type, affiliations, classifactions, and income. Using those features the model would then try to predict if that charity had success.

### Features : Classifaction, Status, Organization Type, Application Type, Affiliation, Use Case, Ask Amount, Income, and Special Considerations
### Target was simply whether or not the charity was successful and if the model could predict it.
### EIN and Name were unecassary and therefore removed.

### For optimization I chose to go with 3 Hidden Layers and 1 output. I went with 3 times the amount of inputs, then reduced that to the same as the number of inputs, and then reduced to the closest whole square root for the input.
### On epoch 100 my accuracy was 75% but it didn't feel like the biggest improvement from the first model. Everything I tried at best gave me 75% but never really got the loss lower than what I had. My biggest concern was if I could reduce loss my accuracy would hopefully go up.
### I ended up binning the asking amounts so they could be one hot encoded, then messing with my layers, through addition of perceptrons and layers and a tweak to activation. This was my most confident result.

### After hearing more about Random Forest Classifiers in Office Hours I would be interested to see how they perform on this dataset.